In [1]:
import MetaTrader5 as mt
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import talib
from talipp.indicators import EMA, SMA, Stoch, DPO
from joblib import dump
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix, classification_report
from own_functions import *
import os
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

mt.initialize()
login = 51708234
password ="4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login,password,server)

symbol = "GBPUSD"
timeframe = mt.TIMEFRAME_D1
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2010, 1, 1), datetime(2024, 10, 18)))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df = ohlc_data[['time', 'open', 'high', 'low', 'close']].copy()


def add_rolling_features(df, window):
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Function to add lag features
def add_lag_features(df, lags):
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Indicators
# Calculate indicators
df['RSI_14'] = talib.RSI(df['close'], timeperiod=14)

df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)

df['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0.02, maximum=0.2)

df['BB_upper'], df['BB_middle'], df['BB_lower'] = talib.BBANDS(df['close'],
                                                               timeperiod=20,
                                                               nbdevup=2,
                                                               nbdevdn=2,
                                                               matype=0)
df['BB_width'] = df['BB_upper'] - df['BB_lower']

df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['close'],
                                                            fastperiod=12,
                                                            slowperiod=26,
                                                            signalperiod=9)

df['CCI_14'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)

df = add_rolling_features(df, window=5)
df = add_lag_features(df, lags=[1, 2, 3, 4, 5])

df = df.dropna().reset_index(drop=True)

# Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

# Dropping NaN values and resetting index
df = df.dropna().reset_index(drop=True)

#csv_file_path = 'EURUSD_D1_2010to101024.csv'  # Specify your desired path
#df.to_csv(csv_file_path, index=False)

StopLoss = 1
TakeProfit = 2
BreakEvenRatio=StopLoss/(StopLoss+TakeProfit)
label_data(df,[StopLoss],[TakeProfit],80,symbol,False)

Mean Candle: 0.011257984852583164


In [2]:
# Calculate total number of 1s in b_flag and s_flag columns
total_b_flags = df['b_flag'].sum()
total_s_flags = df['s_flag'].sum()

# Total number of rows in the DataFrame
total_rows = len(df)

# Calculate counts in segments of complete 100% data
count_100_b_flags = total_b_flags
count_100_s_flags = total_s_flags

# Calculate counts in intervals of 10%
interval_counts = []
for i in range(0, 101, 10):
    start_idx = int(i / 100 * total_rows)
    end_idx = int((i + 10) / 100 * total_rows)
    
    interval_b_flags = df['b_flag'].iloc[start_idx:end_idx].sum()
    interval_s_flags = df['s_flag'].iloc[start_idx:end_idx].sum()
    
    interval_counts.append((f'{i}% - {i+10}%', interval_b_flags, interval_s_flags))

# Print results
print("Total number of 1s:")
print(f"b_flag: {total_b_flags}")
print(f"s_flag: {total_s_flags}")

print("\nCounts in segments of 100% data:")
print(f"b_flag: {count_100_b_flags}")
print(f"s_flag: {count_100_s_flags}")

print("\nCounts in intervals of 10%:")
for interval, count_b, count_s in interval_counts:
    print(f"{interval}: b_flag={count_b}, s_flag={count_s}")

Total number of 1s:
b_flag: 1213
s_flag: 1204

Counts in segments of 100% data:
b_flag: 1213
s_flag: 1204

Counts in intervals of 10%:
0% - 10%: b_flag=113, s_flag=123
10% - 20%: b_flag=136, s_flag=122
20% - 30%: b_flag=137, s_flag=122
30% - 40%: b_flag=96, s_flag=148
40% - 50%: b_flag=131, s_flag=103
50% - 60%: b_flag=134, s_flag=120
60% - 70%: b_flag=118, s_flag=127
70% - 80%: b_flag=135, s_flag=122
80% - 90%: b_flag=96, s_flag=149
90% - 100%: b_flag=117, s_flag=68
100% - 110%: b_flag=0, s_flag=0


In [ ]:
# Feature extraction
df.drop(columns=['s_flag'], inplace=True)

selected_signal_1 = 'WILLR_15'
df_melted_1 = df[['time', selected_signal_1]].copy()
df_melted_1["Symbols"] = symbol

df_rolled_1 = roll_time_series(df_melted_1, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X1 = extract_features(df_rolled_1.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_1, 
                      impute_function=impute, show_warnings=False)

X1 = X1.set_index(X1.index.map(lambda x: x[1]), drop=True)
X1.index.name = "time"
X1 = X1.dropna()

selected_signal_2 = 'WILLR_42'
df_melted_2 = df[['time', selected_signal_2]].copy()
df_melted_2["Symbols"] = symbol

df_rolled_2 = roll_time_series(df_melted_2, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled_2.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_2, 
                      impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2 = X2.dropna()

selected_signal_3 = 'RSI_14'
df_melted_3 = df[['time', selected_signal_3]].copy()
df_melted_3["Symbols"] = symbol

df_rolled_3 = roll_time_series(df_melted_3, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X3 = extract_features(df_rolled_3.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_3, 
                      impute_function=impute, show_warnings=False)

X3 = X3.set_index(X3.index.map(lambda x: x[1]), drop=True)
X3.index.name = "time"
X3 = X3.dropna()

selected_signal_4 = 'MACD_hist'
df_melted_4 = df[['time', selected_signal_4]].copy()
df_melted_4["Symbols"] = symbol

df_rolled_4 = roll_time_series(df_melted_4, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X4 = extract_features(df_rolled_4.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_4, 
                      impute_function=impute, show_warnings=False)

X4 = X4.set_index(X4.index.map(lambda x: x[1]), drop=True)
X4.index.name = "time"
X4 = X4.dropna()

selected_signal_5 = 'CCI_14'
df_melted_5 = df[['time', selected_signal_4]].copy()
df_melted_5["Symbols"] = symbol

df_rolled_5 = roll_time_series(df_melted_5, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X5 = extract_features(df_rolled_5.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_5, 
                      impute_function=impute, show_warnings=False)

X5 = X5.set_index(X5.index.map(lambda x: x[1]), drop=True)
X5.index.name = "time"
X5 = X5.dropna()

X = pd.concat([X1, X2], axis=1, join='inner')
X = X.dropna()

# Align indices
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df[df.index.isin(X.index)]

X = pd.concat([X, df], axis=1, join='inner')

# Ensure b_flag is at the end after feature selection
X_df = select_features(X, X['b_flag'])
X_df = X_df[[col for col in X_df if col != 'b_flag'] + ['b_flag']]

correlation_matrix = X_df.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_correlation_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.8)]
X_df = X_df.drop(columns=high_correlation_features)


original_index = X_df.index
shifted_X_df = X_df.shift(periods=1, axis=0)  # This shifts both features and target
shifted_X_df.index = original_index  # Keep the original index
X_df = shifted_X_df.dropna()



# Get the list of selected feature names
selected_feature_names_X = list(X_df.columns)

# Combine lists if you need a single list for all selected features

print(selected_feature_names_X )

Feature Extraction: 100%|██████████| 30/30 [01:16<00:00,  2.56s/it]


['WILLR_42__mean_second_derivative_central', 'WILLR_15', 'WILLR_15__fft_coefficient__attr_"real"__coeff_10', 'WILLR_15__fft_coefficient__attr_"real"__coeff_9', 'WILLR_15__agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"max"', 'WILLR_15__fft_coefficient__attr_"imag"__coeff_7', 'WILLR_15__fft_coefficient__attr_"imag"__coeff_6', 'WILLR_15__fft_coefficient__attr_"imag"__coeff_5', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2', 'WILLR_42__fft_coefficient__attr_"angle"__coeff_7', 'WILLR_15__fft_coefficient__attr_"imag"__coeff_8', 'WILLR_15__fft_coefficient__attr_"real"__coeff_8', 'WILLR_42__agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"min"', 'WILLR_42__agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"max"', 'WILLR_42__agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"', 'WILLR_15__fft_coefficient__attr_"angle"__coeff_6', 'WILLR_15__fft_coefficient__attr_"angle"__coeff_7', 'WILLR_15__fft_coefficient__attr_"imag"__coeff_4', 'WILLR_42__fft_co

In [4]:
sum_fp = 0
sum_tp = 0

# Split into train and test sets
split = int(0.90 * len(X_df))  # Use the feature-engineered X_df, not df

train_data, test_data = X_df.iloc[:split], X_df.iloc[split:]


# Train data
# Ensure correct feature and target selection:
x_train = train_data.iloc[:, :-1].values  # Features are all columns except the last one (s_flag)
y_train = train_data['b_flag'].values  # Target is the s_flag column

x_test = test_data.iloc[:, :-1].values
y_test = test_data['b_flag'].values

# Scale Data
sc_mt = StandardScaler()

x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)

os.makedirs('GBPUSD_D1_3112buyfinal', exist_ok=True)
# Save the scaler
dump(sc_mt, 'GBPUSD_D1_3112buyfinal/scaler.joblib')

# Hyperparameters
n_estimators = 120
class_weight = {0: 6, 1: 1}
max_features = 'sqrt'
random_state = 8

# Initialize RandomForestClassifier
rf_classifier_mt = RandomForestClassifier(
    n_estimators=n_estimators,
    class_weight=class_weight,
    max_features=max_features,
    random_state=random_state
)

# Train the model
rf_classifier_mt.fit(x_train, y_train)

# Save the model
dump(rf_classifier_mt, 'GBPUSD_D1_3112buyfinal/model.joblib')

# Predict on test set
y_pred = rf_classifier_mt.predict(x_test)

# Print confusion matrix
print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

false_positives = conf_matrix[0][1]
true_positives = conf_matrix[1][1]

sum_fp += false_positives
sum_tp += true_positives

# Print additional metrics
precision = precision_score(y_test, y_pred)

print('WIN/LOSS-Diff:', round(100 * (precision - BreakEvenRatio), 2), '%')
print('sum_fp:', sum_fp)
print('sum_tp:', sum_tp)
print('precision:', precision)
print('Ratio total:', round(100 * (sum_tp / (sum_fp + sum_tp)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')


Confusion Matrix:
[[192  61]
 [ 65  52]]
WIN/LOSS-Diff: 12.68 %
sum_fp: 61
sum_tp: 52
precision: 0.46017699115044247
Ratio total: 46.02
BreakEvenRatio: 0.33
____________________________________________________________________________________________________________________________


In [5]:
import json
feature_names = X_df.columns
with open('GBPUSD_D1_3112buyfinal/feature_names.json', 'w') as f:
    json.dump(list(feature_names), f)


In [38]:
# Convert index to datetime without 'unit' since the format is already date strings
X_df.index = pd.to_datetime(X_df.index)

# Format datetime to the desired string format
X_df.index = X_df.index.strftime('%Y-%m-%d %H:%M:%S')

# Creating a DataFrame for predictions with the correct index
df_pred = pd.DataFrame(index=X_df.iloc[split:].index)  # No need for split+1
df_pred['prediction'] = y_pred

# Save to CSV
df_pred.to_csv('predGBPUSD_D1_3112buy.csv')


In [41]:
import pandas as pd
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import logging
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

# Logging configuration
logging.basicConfig(filename='backtest.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# MetaTrader 5 initialization
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch historical OHLC data from MetaTrader 5
def fetch_ohlc_data(symbol, timeframe, start_date, end_date):
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    if data is None or len(data) == 0:
        logging.error(f"Failed to fetch data for {symbol}")
        return None
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'tick_volume': 'Volume'}, inplace=True)
    return ohlc_data[['time', 'Open', 'High', 'Low', 'Close', 'Volume']]  # Include Volume

# Load and align prediction data
def load_and_align_data(ohlc_data, prediction_file):
    try:
        predictions = pd.read_csv(prediction_file, parse_dates=['time'])
        if 'prediction' not in predictions.columns:
            logging.error(f"'prediction' column not found in {prediction_file}")
            return None
    except Exception as e:
        logging.error(f"Error loading prediction file: {e}")
        return None
    # Merge predictions with OHLC data
    ohlc_data = ohlc_data.merge(predictions[['time', 'prediction']], on='time', how='left')
    ohlc_data['prediction'] = ohlc_data['prediction'].fillna(0)  # Fill missing predictions with 0
    ohlc_data['prediction'] = ohlc_data['prediction'].shift(1)  # Shift predictions to next date
    return ohlc_data

# Backtesting strategy for Buy or Sell
class PredictionStrategy(Strategy):
    risk_reward_ratio = (2, 3)  # Default risk-reward ratio
    signal_type = 'Buy'  # Default signal type
    mean_candle_size = 0.0105  # Default mean candle size

    def init(self):
        # Mean candle size now taken from strategy properties set during strategy initialization
        pass

    def next(self):
        entry_price = self.data.Close[-1]
        risk_part, reward_part = self.risk_reward_ratio
        # Buy signal
        if self.data.prediction[-1] == 1 and self.signal_type == 'Buy':
            sl_price = entry_price - self.mean_candle_size * risk_part
            tp_price = entry_price + self.mean_candle_size * reward_part
            self.buy(sl=sl_price, tp=tp_price)
        # Sell signal
        elif self.data.prediction[-1] == 1 and self.signal_type == 'Sell':
            sl_price = entry_price + self.mean_candle_size * risk_part
            tp_price = entry_price - self.mean_candle_size * reward_part
            self.sell(sl=sl_price, tp=tp_price)

# Function to perform backtesting and save stats/plot
def run_backtest(ohlc_data, strategy_class, risk_reward_ratio, pair_name, signal_type, mean_candle_size):
    strategy_class.risk_reward_ratio = risk_reward_ratio
    strategy_class.signal_type = signal_type
    strategy_class.mean_candle_size = mean_candle_size  # Set mean candle size for the strategy
    bt = Backtest(ohlc_data.set_index('time'), strategy_class, cash=10000, commission=.0003)
    stats = bt.run()
    plt.figure(figsize=(10, 6))
    bt.plot()
    plt.title(f'Backtest for {pair_name} - {signal_type}')
    #plt.savefig(f'backtest_plot_{pair_name}_{signal_type}.png')
    plt.close()
    stats_df = pd.DataFrame([stats])
    #stats_df.to_csv(f'backtest_stats_{pair_name}_{signal_type}.csv', index=False)
    return stats

# Main function to fetch OHLC data, align it with prediction data, and run backtest for each buy/sell
def main():
    config = {
        'login': 51988090,
        'password': '1fMdV52$74EOcw',
        'server': 'ICMarketsEU-Demo',
        'GBPUSD': {
            'symbol': 'GBPUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.011266204280682732,
            'buy_prediction_file': 'predGBPUSD_D1_3112buy.csv',
            'buy_risk_reward_ratio': (1, 2),
        }
    }

    init_mt5_connection(config['login'], config['password'], config['server'])
    utc_from = datetime(2023, 5, 10, tzinfo=pytz.utc)
    utc_to = datetime(2024, 10, 7, tzinfo=pytz.utc)

    for pair_name, pair_config in config.items():
        if pair_name in ['login', 'password', 'server']:
            continue
        logging.info(f"Processing {pair_name}...")
        ohlc_data = fetch_ohlc_data(pair_config['symbol'], pair_config['timeframe'], utc_from, utc_to)
        if ohlc_data is None:
            continue

        if pair_config['buy_prediction_file']:
            ohlc_data_with_predictions = load_and_align_data(ohlc_data, pair_config['buy_prediction_file'])
            if ohlc_data_with_predictions is not None:
                stats = run_backtest(ohlc_data_with_predictions, PredictionStrategy, pair_config['buy_risk_reward_ratio'], pair_name, 'Buy', pair_config['mean_candle_size'])
                print(f"Backtest results for {pair_name} - Buy:\n", stats)
        if pair_config.get('sell_prediction_file'):
            ohlc_data_with_predictions = load_and_align_data(ohlc_data, pair_config['sell_prediction_file'])
            if ohlc_data_with_predictions is not None:
                stats = run_backtest(ohlc_data_with_predictions, PredictionStrategy, pair_config['sell_risk_reward_ratio'], pair_name, 'Sell', pair_config['mean_candle_size'])
                print(f"Backtest results for {pair_name} - Sell:\n", stats)

    mt5.shutdown()

if __name__ == "__main__":
    main()


INFO:bokeh.io.state:Session output file 'PredictionStrategy.html' already exists, will be overwritten.


Connected to MetaTrader 5
Backtest results for GBPUSD - Buy:
 Start                     2023-05-10 00:00:00
End                       2024-10-07 00:00:00
Duration                    516 days 00:00:00
Exposure Time [%]                   79.019074
Equity Final [$]                 10893.397407
Equity Peak [$]                  11153.230233
Return [%]                           8.933974
Buy & Hold Return [%]                3.626225
Return (Ann.) [%]                    6.051828
Volatility (Ann.) [%]                6.845215
Sharpe Ratio                         0.884096
Sortino Ratio                        1.459277
Calmar Ratio                         0.943356
Max. Drawdown [%]                   -6.415208
Avg. Drawdown [%]                    -1.37939
Max. Drawdown Duration      319 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                   26
Win Rate [%]                        46.153846
Best Trade [%]                       1.840256
Worst Trade [%]   

In [86]:
import pandas as pd
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import logging
from datetime import datetime
import pytz
import sys
import matplotlib.pyplot as plt

# Logging configuration
logging.basicConfig(filename='backtest.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# MetaTrader 5 initialization
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch historical OHLC data from MetaTrader 5
def fetch_ohlc_data(symbol, timeframe, start_date, end_date):
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    if data is None or len(data) == 0:
        logging.error(f"Failed to fetch data for {symbol}")
        return None
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'tick_volume': 'Volume'}, inplace=True)
    return ohlc_data[['time', 'Open', 'High', 'Low', 'Close', 'Volume']]  # Include Volume, even if it's not mandatory

# Load and align prediction data
def load_and_align_data(ohlc_data, prediction_file):
    try:
        predictions = pd.read_csv(prediction_file, parse_dates=['time'])
        if 'prediction' not in predictions.columns:
            logging.error(f"'prediction' column not found in {prediction_file}")
            return None
    except Exception as e:
        logging.error(f"Error loading prediction file: {e}")
        return None

    # Merge predictions with OHLC data
    ohlc_data = ohlc_data.merge(predictions[['time', 'prediction']], on='time', how='left')
    ohlc_data['prediction'] = ohlc_data['prediction'].fillna(0)  # Fill missing predictions with 0
    ohlc_data['prediction'] = ohlc_data['prediction'].shift(1)  # Shift predictions by one day to start trade on the next date
    return ohlc_data

# Backtesting strategy for Buy or Sell
class PredictionStrategy(Strategy):
    risk_reward_ratio = (2, 3)  # Default risk-reward ratio
    signal_type = 'Buy'  # Default signal type
    mean_candle_size = 0  # Mean candle size based on historical data

    def init(self):
        # Calculate mean candle size based on historical data
        self.mean_candle_size = 0.0105

    def next(self):
        entry_price = self.data.Close[-1]
        risk_part, reward_part = self.risk_reward_ratio

        # Buy signal
        if self.data.prediction[-1] == 1 and self.signal_type == 'Buy':
            sl_price = entry_price - self.mean_candle_size * risk_part
            tp_price = entry_price + self.mean_candle_size * reward_part
            self.buy(sl=sl_price, tp=tp_price)

        # Sell signal
        elif self.data.prediction[-1] == 1 and self.signal_type == 'Sell':
            sl_price = entry_price + self.mean_candle_size * risk_part
            tp_price = entry_price - self.mean_candle_size * reward_part
            self.sell(sl=sl_price, tp=tp_price)

# Function to perform backtesting and save stats/plot
def run_backtest(ohlc_data, strategy_class, risk_reward_ratio, pair_name, signal_type):
    strategy_class.risk_reward_ratio = risk_reward_ratio  # Set specific risk-reward ratio for each pair
    strategy_class.signal_type = signal_type  # Set the signal type (Buy or Sell)
    bt = Backtest(ohlc_data.set_index('time'), strategy_class, cash=10000, commission=.0003
                  )
    stats = bt.run()

    # Generate and save the backtest plot
    plt.figure(figsize=(10, 6))
    bt.plot()  # This generates the plot using backtesting.py's internal plot function
    plt.draw()  # Make sure the plot is rendered properly
    plt.pause(0.1)  # Pause to ensure rendering before saving
    plt.title(f'Backtest for {pair_name} - {signal_type}')  # Add pair name and signal type to the title
    plt.savefig(f'backtest_plot_{pair_name}_{signal_type}.png')  # Save the plot manually using matplotlib
    plt.close()  # Close the plot to avoid displaying it in the environment

    # Save backtest stats as CSV
    stats_df = pd.DataFrame([stats])
    stats_df.to_csv(f'backtest_stats_{pair_name}_{signal_type}.csv', index=False)

    return stats


# Main function to fetch OHLC data, align it with prediction data, and run backtest for each buy/sell
def main():
    # Configuration for MetaTrader 5 connection
    config = {
        'login': 51988090,
        'password': '1fMdV52$74EOcw',
        'server': 'ICMarketsEU-Demo'
    }

    # Initialize MetaTrader 5 connection
    init_mt5_connection(config['login'], config['password'], config['server'])

    # Define the time period for backtesting
    utc_from = datetime(2023, 5, 9, tzinfo=pytz.utc)
    utc_to = datetime(2024, 10, 7, tzinfo=pytz.utc)

    # Currency pair configurations
    currency_pairs = {
        'USDCAD': {
            'symbol': 'USDCAD',
            'timeframe': mt5.TIMEFRAME_D1,
            'buy_prediction_file': 'predUSDCAD_D1_3112buy.csv',
            'buy_risk_reward_ratio': (1, 1),  # EURUSD Buy Risk-Reward Ratio
            'sell_risk_reward_ratio': (1, 2),  # EURUSD Sell Risk-Reward Ratio
        }
    }

    # Loop through each currency pair and perform backtest
    for pair_name, pair_config in currency_pairs.items():
        logging.info(f"Processing {pair_name}...")

        # Fetch OHLC data
        ohlc_data = fetch_ohlc_data(pair_config['symbol'], pair_config['timeframe'], utc_from, utc_to)
        if ohlc_data is None:
            logging.error(f"Skipping {pair_name} due to missing OHLC data")
            continue

        # Backtest Buy predictions if the file exists
        if pair_config['buy_prediction_file']:
            ohlc_data_with_predictions = load_and_align_data(ohlc_data, pair_config['buy_prediction_file'])
            if ohlc_data_with_predictions is not None:
                stats = run_backtest(ohlc_data_with_predictions, PredictionStrategy, pair_config['buy_risk_reward_ratio'], pair_name, 'Buy')
                print(f"Backtest results for {pair_name} - Buy:\n", stats)
            else:
                logging.error(f"Skipping {pair_name} Buy due to prediction data issue")

        # Backtest Sell predictions if the file exists
        if pair_config.get('sell_prediction_file'):
            ohlc_data_with_predictions = load_and_align_data(ohlc_data, pair_config['sell_prediction_file'])
            if ohlc_data_with_predictions is not None:
                stats = run_backtest(ohlc_data_with_predictions, PredictionStrategy, pair_config['sell_risk_reward_ratio'], pair_name, 'Sell')
                print(f"Backtest results for {pair_name} - Sell:\n", stats)
            else:
                logging.error(f"Skipping {pair_name} Sell due to prediction data issue")

    # Shutdown MetaTrader 5 connection after backtesting
    mt5.shutdown()

if __name__ == "__main__":
    main()

INFO:bokeh.io.state:Session output file 'PredictionStrategy.html' already exists, will be overwritten.


Connected to MetaTrader 5


<Figure size 1000x600 with 0 Axes>

Backtest results for USDCAD - Buy:
 Start                     2023-05-09 00:00:00
End                       2024-10-07 00:00:00
Duration                    517 days 00:00:00
Exposure Time [%]                   86.684783
Equity Final [$]                 48802.566535
Equity Peak [$]                  94408.056452
Return [%]                         388.025665
Buy & Hold Return [%]                1.774547
Return (Ann.) [%]                  196.096697
Volatility (Ann.) [%]            19362.497774
Sharpe Ratio                         0.010128
Sortino Ratio                        1.170442
Calmar Ratio                         2.000718
Max. Drawdown [%]                  -98.013168
Avg. Drawdown [%]                  -38.732644
Max. Drawdown Duration      369 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                                   99
Win Rate [%]                        64.646465
Best Trade [%]                       0.922453
Worst Trade [%]                     -0.81745

In [65]:
import pandas as pd
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import logging
from datetime import datetime
import pytz
import sys
import matplotlib.pyplot as plt

# Logging configuration
logging.basicConfig(filename='backtest.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# MetaTrader 5 initialization
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch historical OHLC data from MetaTrader 5
def fetch_ohlc_data(symbol, timeframe, start_date, end_date):
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    if data is None or len(data) == 0:
        logging.error(f"Failed to fetch data for {symbol}")
        return None
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'tick_volume': 'Volume'}, inplace=True)
    return ohlc_data[['time', 'Open', 'High', 'Low', 'Close', 'Volume']]  # Include Volume, even if it's not mandatory

# Load and align prediction data
def load_and_align_data(ohlc_data, buy_prediction_file, sell_prediction_file):
    try:
        buy_predictions = pd.read_csv(buy_prediction_file, parse_dates=['time'])
        sell_predictions = pd.read_csv(sell_prediction_file, parse_dates=['time'])
        if 'prediction' not in buy_predictions.columns or 'prediction' not in sell_predictions.columns:
            logging.error(f"'prediction' column not found in one of the prediction files")
            return None
    except Exception as e:
        logging.error(f"Error loading prediction file: {e}")
        return None

    # Merge predictions with OHLC data
    ohlc_data = ohlc_data.merge(buy_predictions[['time', 'prediction']], on='time', how='left', suffixes=('', '_buy'))
    ohlc_data = ohlc_data.merge(sell_predictions[['time', 'prediction']], on='time', how='left', suffixes=('', '_sell'))
    ohlc_data['prediction_buy'] = ohlc_data['prediction'].fillna(0)
    ohlc_data['prediction_sell'] = ohlc_data['prediction_sell'].fillna(0)
    ohlc_data['prediction_buy'] = ohlc_data['prediction_buy'].shift(1)  # Shift predictions by one day to start trade on the next date
    ohlc_data['prediction_sell'] = ohlc_data['prediction_sell'].shift(1)
    ohlc_data.drop(columns=['prediction'], inplace=True)
    return ohlc_data

# Backtesting strategy for Buy or Sell
class PredictionStrategy(Strategy):
    risk_reward_ratio_buy = (2, 3)  # Default risk-reward ratio for Buy
    risk_reward_ratio_sell = (2, 3)  # Default risk-reward ratio for Sell
    mean_candle_size = 0  # Mean candle size based on historical data

    def init(self):
        # Calculate mean candle size based on historical data
        self.mean_candle_size = (self.data.High - self.data.Low).mean()

    def next(self):
        entry_price = self.data.Close[-1]
        risk_part_buy, reward_part_buy = self.risk_reward_ratio_buy
        risk_part_sell, reward_part_sell = self.risk_reward_ratio_sell

        # Check if both Buy and Sell are predicted, in which case no trade should be taken
        if self.data.prediction_buy[-1] == 1 and self.data.prediction_sell[-1] == 1:
            print(f"Skipping trade on {self.data.index[-1]} due to both Buy and Sell signals")
            return

        # Buy signal
        if self.data.prediction_buy[-1] == 1:
            sl_price = entry_price - self.mean_candle_size * risk_part_buy
            tp_price = entry_price + self.mean_candle_size * reward_part_buy
            print(f"Attempting to Buy on {self.data.index[-1]} at {entry_price} with SL={sl_price} TP={tp_price}")
            self.buy(sl=sl_price, tp=tp_price)

        # Sell signal
        elif self.data.prediction_sell[-1] == 1:
            sl_price = entry_price + self.mean_candle_size * risk_part_sell
            tp_price = entry_price - self.mean_candle_size * reward_part_sell
            print(f"Attempting to Sell on {self.data.index[-1]} at {entry_price} with SL={sl_price} TP={tp_price}")
            self.sell(sl=sl_price, tp=tp_price)


# Function to perform backtesting and save stats/plot
def run_backtest(ohlc_data, strategy_class, risk_reward_ratio_buy, risk_reward_ratio_sell, pair_name):
    strategy_class.risk_reward_ratio_buy = risk_reward_ratio_buy  # Set specific risk-reward ratio for Buy
    strategy_class.risk_reward_ratio_sell = risk_reward_ratio_sell  # Set specific risk-reward ratio for Sell
    bt = Backtest(ohlc_data.set_index('time'), strategy_class, cash=10000, commission=.0003)
    stats = bt.run()

    # Generate and save the backtest plot
    plt.figure(figsize=(10, 6))
    bt.plot()  # This generates the plot using backtesting.py's internal plot function
    plt.draw()  # Make sure the plot is rendered properly
    plt.pause(0.1)  # Pause to ensure rendering before saving
    plt.title(f'Backtest for {pair_name}')  # Add pair name to the title
    plt.savefig(f'backtest_plot_{pair_name}.png')  # Save the plot manually using matplotlib
    plt.close()  # Close the plot to avoid displaying it in the environment

    # Save backtest stats as CSV
    stats_df = pd.DataFrame([stats])
    stats_df.to_csv(f'backtest_stats_{pair_name}.csv', index=False)

    return stats

# Main function to fetch OHLC data, align it with prediction data, and run backtest
def main():
    # Configuration for MetaTrader 5 connection
    config = {
        'login': 51988090,
        'password': '1fMdV52$74EOcw',
        'server': 'ICMarketsEU-Demo'
    }

    # Initialize MetaTrader 5 connection
    init_mt5_connection(config['login'], config['password'], config['server'])

    # Define the time period for backtesting
    utc_from = datetime(2019, 12, 3, tzinfo=pytz.utc)
    utc_to = datetime(2024, 10, 7, tzinfo=pytz.utc)

    # Currency pair configurations
    currency_pairs = {
        'GBPUSD': {
            'symbol': 'GBPUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'buy_prediction_file': 'GBPUSD_D1_3112_Buy.csv',
            'sell_prediction_file': 'predictGBPUSD_D1_3112sell.csv',
            'buy_risk_reward_ratio': (1, 2),  # GBPUSD Buy Risk-Reward Ratio
            'sell_risk_reward_ratio': (1, 2),  # GBPUSD Sell Risk-Reward Ratio
        },
        'USDCAD': {
            'symbol': 'USDCAD',
            'timeframe': mt5.TIMEFRAME_D1,
            'buy_prediction_file': 'predictUSDCAD_D1Buy.csv',
            'sell_prediction_file': 'predictUSDCAD_D1Sell.csv',
            'buy_risk_reward_ratio': (2, 3),  # USDCAD Buy Risk-Reward Ratio
            'sell_risk_reward_ratio': (2, 3),  # USDCAD Sell Risk-Reward Ratio
        },
        'EURUSD': {
            'symbol': 'EURUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'buy_prediction_file': 'predictEURUSD_D1_3112buy.csv',
            'sell_prediction_file': 'predict_EURUSD_D1_3112_Sell.csv',
            'buy_risk_reward_ratio': (2, 3),  # EURUSD Buy Risk-Reward Ratio
            'sell_risk_reward_ratio': (1, 2),  # EURUSD Sell Risk-Reward Ratio
        }
    }

    # Loop through each currency pair and perform backtest
    for pair_name, pair_config in currency_pairs.items():
        logging.info(f"Processing {pair_name}...")

        # Fetch OHLC data
        ohlc_data = fetch_ohlc_data(pair_config['symbol'], pair_config['timeframe'], utc_from, utc_to)
        if ohlc_data is None:
            logging.error(f"Skipping {pair_name} due to missing OHLC data")
            continue

        # Load and align data for both Buy and Sell predictions
        ohlc_data_with_predictions = load_and_align_data(ohlc_data, pair_config['buy_prediction_file'], pair_config['sell_prediction_file'])
        if ohlc_data_with_predictions is not None:
            stats = run_backtest(ohlc_data_with_predictions, PredictionStrategy, pair_config['buy_risk_reward_ratio'], pair_config['sell_risk_reward_ratio'], pair_name)
            print(f"Backtest results for {pair_name}:", stats)
        else:
            logging.error(f"Skipping {pair_name} due to prediction data issue")

    # Shutdown MetaTrader 5 connection after backtesting
    mt5.shutdown()

if __name__ == "__main__":
    main()

INFO:bokeh.io.state:Session output file 'PredictionStrategy.html' already exists, will be overwritten.


Connected to MetaTrader 5
Attempting to Sell on 2021-08-30 00:00:00 at 1.37571 with SL=1.3865268600953895 TP=1.354076279809221
Attempting to Sell on 2021-09-08 00:00:00 at 1.37699 with SL=1.3878068600953894 TP=1.355356279809221
Attempting to Sell on 2021-09-09 00:00:00 at 1.38362 with SL=1.3944368600953896 TP=1.361986279809221
Attempting to Sell on 2021-09-16 00:00:00 at 1.37879 with SL=1.3896068600953895 TP=1.357156279809221
Attempting to Sell on 2021-09-20 00:00:00 at 1.3651200000000001 with SL=1.3759368600953896 TP=1.343486279809221
Attempting to Sell on 2021-09-21 00:00:00 at 1.36596 with SL=1.3767768600953896 TP=1.344326279809221
Attempting to Sell on 2021-09-22 00:00:00 at 1.3616 with SL=1.3724168600953894 TP=1.339966279809221
Attempting to Sell on 2021-09-23 00:00:00 at 1.37196 with SL=1.3827768600953896 TP=1.350326279809221
Attempting to Sell on 2021-09-24 00:00:00 at 1.36672 with SL=1.3775368600953894 TP=1.345086279809221
Attempting to Sell on 2021-09-28 00:00:00 at 1.3534 wit

<Figure size 1000x600 with 0 Axes>

INFO:bokeh.io.state:Session output file 'PredictionStrategy.html' already exists, will be overwritten.


Backtest results for GBPUSD: Start                     2019-12-03 00:00:00
End                       2024-10-07 00:00:00
Duration                   1770 days 00:00:00
Exposure Time [%]                   51.987281
Equity Final [$]                 10728.201172
Equity Peak [$]                   12752.00073
Return [%]                           7.282012
Buy & Hold Return [%]                0.695032
Return (Ann.) [%]                    1.418011
Volatility (Ann.) [%]                7.355279
Sharpe Ratio                         0.192788
Sortino Ratio                        0.290747
Calmar Ratio                          0.07309
Max. Drawdown [%]                  -19.400968
Avg. Drawdown [%]                    -2.93581
Max. Drawdown Duration      698 days 00:00:00
Avg. Drawdown Duration       61 days 00:00:00
# Trades                                  188
Win Rate [%]                        41.489362
Best Trade [%]                       1.969382
Worst Trade [%]                     -1.296267
Avg. 

<Figure size 1000x600 with 0 Axes>

Backtest results for USDCAD: Start                     2019-12-03 00:00:00
End                       2024-10-07 00:00:00
Duration                   1770 days 00:00:00
Exposure Time [%]                   82.750397
Equity Final [$]                  8275.151117
Equity Peak [$]                  11143.534942
Return [%]                         -17.248489
Buy & Hold Return [%]                2.458178
Return (Ann.) [%]                   -3.721561
Volatility (Ann.) [%]                6.399295
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -27.898283
Avg. Drawdown [%]                   -1.967253
Max. Drawdown Duration     1223 days 00:00:00
Avg. Drawdown Duration       67 days 00:00:00
# Trades                                  233
Win Rate [%]                        45.922747
Best Trade [%]                       2.112849
Worst Trade [%]                     -1.580278
Avg. 